In [1]:
def random_float(min=0, max=9):
    import random
    entier = random.randint(min, max)
    float_ = random.random()
    return entier+float_

In [2]:
class Position():
    def __init__(self, longitude, lattitude):
        self.longitude = longitude
        self.lattitude = lattitude


In [3]:
class Logger():
    
    def __init__(self, path, bufferlength=100, separator="\n", mode="a"):
        import os
        if os.path.exists(path) :
            msg = "Attention : vous allez utiliser un fichier existant !"
            print msg
            if (mode != "a"):
                raise msg
        self.path         = path
        self.logs         = []
        self.bufferlength = 100
        self.separator    = separator

    def clean_msg(self, msg):
        msg = str(msg)
        if msg[-1] != "\n": 
            msg = msg + "\n"
        return msg
    
    def log(self, msg):
        msg = self.clean_msg(msg)
        self.logs.append(msg)
        if self.bufferlength < len(self.logs):
            self.write_to_disk()
    
    def write_to_disk(self):
        with open(self.path, "a") as f:
            f.writelines(self.logs)
        self.logs= []
    
    def __del__(self):
        self.write_to_disk()
                

In [4]:
def get_random_position():
    return Position(random_float(), random_float())

In [5]:
class Velo:
    nb_d_instance =0
    def __init__(self):
        import random
        Velo.nb_d_instance += 1
        self.id             = "velo_%s"%(Velo.nb_d_instance)
        self.performance    = random.random()
    def update_performance(self):
        import numpy as np
        self.performance = np.random.normal(0.75, 0.2)

In [6]:
class Trajet():
    def __init__(self, position_depart, position_arrivee):
        self.depart      = position_depart
        self.destination = position_arrivee
        self.set_position(position_depart)
        self.set_distance() 
        self.set_pas()
        self.avancee_minimale    = 1
        
    def set_position(self, position):
        self.longitude   = position.longitude
        self.lattitude   = position.lattitude
        self.position    = position
        
    def set_pas(self):
        delta_longitude    = self.destination.longitude  - self.longitude
        delta_lattitude    = self.destination.lattitude  - self.lattitude
        self.pas_longitude = delta_longitude / float(10)
        self.pas_lattitude = delta_lattitude / float(10)
        
    def set_distance(self):
        import math
        delta_longitude = self.destination.longitude  - self.longitude
        delta_lattitude = self.destination.lattitude  - self.lattitude
        distance        = math.sqrt(math.pow(delta_longitude, 2) + math.pow(delta_lattitude, 2))
        self.distance   = distance
        
    def next_position(self):
        self.set_distance() 
        if self.distance < self.avancee_minimale:
            self.set_position(self.destination)
            return self.destination
        else:
            new_longitude   = self.longitude + self.pas_longitude
            new_latitude    = self.lattitude + self.pas_lattitude
            new_position    = Position(new_longitude, new_latitude)
            self.set_position(new_position)
            return new_position

In [7]:
class JcDecaux():
    def __init__(self, largeur=10, hauteur=10):
        self.largeur = largeur
        self.hauteur = hauteur
        self.reseau = []
        for lattitude in range(self.largeur):
            for longitude in range(self.hauteur):
                emplacement = Position(lattitude, longitude)
                new_station = Station(emplacement)
                self.reseau.append(new_station)
                
    def get_closest_station(self, position):
        nb_ligne = round(position.lattitude)
        nb_col   = round(position.longitude)
        
        indice   =  (nb_ligne * self.largeur) + (nb_col )
        try:
            closest  = self.reseau[int(indice)]
        except:
                closest  = self.reseau[-1]
        return closest
        
        

In [8]:
class Station:
    nb_d_instance =0
    def __init__(self, position):
        import random
        Station.nb_d_instance +=1
        NB_BORNES_POSSIBLES = [5,10,15,20,30]
        self.position       = position
        self.nb_bornes      = random.choice(NB_BORNES_POSSIBLES)
        self.velos          = [Velo() for i in range(self.nb_bornes)]
        self.id             = Station.nb_d_instance
    
    def has_empty_place(self):
        return len(self.velos) < self.nb_bornes

    def has_velo(self):
        return 0 < len(self.velos)
    def log(self, message):
        print "station_%s : %s"%(self.id, message)
        
    def deposer_un_velo(self, velo):
        if self.has_empty_place():
            velo.update_performance()
            self.velos.append(velo)
            self.log("recupere le velo %s"%velo.id)
            return True
        else:
            self.log("complete")
            return False
    def prendre_un_velo(self):
        if self.has_velo():
            velo = self.velos.pop()
            self.log("donne le velo %s"%velo.id)
            return velo
        else:
            self.log("vide.")
            return False

In [9]:
class Cycliste:
    nb_d_instance =0
    def __init__(self, position_home, position_travail):
        import random
        import numpy as np
        Cycliste.nb_d_instance +=1
        self.id           = Cycliste.nb_d_instance
        self.home         = position_home
        self.travail      = position_travail
        self.est_en_route = False
        self.position     = position_home
        self.destination  = position_travail
        self.sur_velo     = False
        self.velo         = False
        self.set_next_trajet()
        self.sexe         = random.choice(["H", "F"])
        self.age          = random.choice(range(15,85))
        self.sportivite   = np.random.normal(2, 2)
        
        
        
    def est_a_la_maison(self):
        return self.position == self.home
    def est_au_travail(self):
        return self.position == self.travail
    def est_arrive(self):
        return self.position == self.destination
    
    def log(self, message):
        print "cycliste_%s : %s"%(self.id, message)
        
    def trouve_velo(self):
        limite = 10
        while not self.sur_velo and 0 < limite :
            limite         = limite - 1
            station_proche = reseau.get_closest_station(self.position)
            velo           = station_proche.prendre_un_velo()
            self.log("essaie station : %s "%station_proche.id)
            self.position  = station_proche.position 
            if velo:
                self.log("trouve velo %s sur station %s"%(velo.id, station_proche.id))
                self.sur_velo = True
                self.velo     = velo
                self.set_next_trajet()
                
    def rendre_velo(self):
        limite = 10
        while self.sur_velo and 0 < limite :
            limite         = limite - 1
            station_proche = reseau.get_closest_station(self.position)
            velo_rendu = station_proche.deposer_un_velo(self.velo)
            self.log("essaie de rendre sur la station : %s "%station_proche.id)
            self.position  = station_proche.position 
            if velo_rendu:
                self.log( "velo rendu sur la station sur : %s"%station_proche.id)
                self.sur_velo = False
                self.velo     = False
                self.set_next_trajet()
                
    def set_next_trajet(self):
        if self.est_a_la_maison():
            self.log("next trajet = travail")
            self.destination = self.travail
        if self.est_au_travail():
            self.log("next trajet = maison")
            self.destination = self.home
        self.trajet = Trajet(self.position, self.destination)
    
    
    def avancer(self):
        if self.sur_velo:
            self.log( "est sur un velo")
            if self.est_arrive():
                self.log( "est arrive")
                self.rendre_velo()
                self.set_next_trajet()
            else:
                self.position = self.trajet.next_position()
                self.log( "va a la prochaine position : %s "%self.position.__dict__)
        else:
            self.log( "va chercher un velo")
            self.trouve_velo()
            
        

In [10]:
class Ville:
    def __init__(self, nb_de_cyclistes):
        self.cyclistes = [Cycliste(get_random_position(), get_random_position()) for i in range(nb_de_cyclistes)]
        
    def reveil(self):
        map(lambda cycliste : cycliste.avancer(), self.cyclistes)
        

In [11]:
reseau = JcDecaux(10,10)


In [12]:
paris = Ville(1000)


cycliste_1 : next trajet = travail
cycliste_2 : next trajet = travail
cycliste_3 : next trajet = travail
cycliste_4 : next trajet = travail
cycliste_5 : next trajet = travail
cycliste_6 : next trajet = travail
cycliste_7 : next trajet = travail
cycliste_8 : next trajet = travail
cycliste_9 : next trajet = travail
cycliste_10 : next trajet = travail
cycliste_11 : next trajet = travail
cycliste_12 : next trajet = travail
cycliste_13 : next trajet = travail
cycliste_14 : next trajet = travail
cycliste_15 : next trajet = travail
cycliste_16 : next trajet = travail
cycliste_17 : next trajet = travail
cycliste_18 : next trajet = travail
cycliste_19 : next trajet = travail
cycliste_20 : next trajet = travail
cycliste_21 : next trajet = travail
cycliste_22 : next trajet = travail
cycliste_23 : next trajet = travail
cycliste_24 : next trajet = travail
cycliste_25 : next trajet = travail
cycliste_26 : next trajet = travail
cycliste_27 : next trajet = travail
cycliste_28 : next trajet = travail
c

In [13]:
for i in range(10):
    paris.reveil()
    for s in reseau.reseau:
        print len(s.velos), "velos <", s.nb_bornes, "?", s.has_empty_place()

cycliste_1 : va chercher un velo
station_88 : donne le velo velo_1295
cycliste_1 : essaie station : 88 
cycliste_1 : trouve velo velo_1295 sur station 88
cycliste_2 : va chercher un velo
station_98 : donne le velo velo_1460
cycliste_2 : essaie station : 98 
cycliste_2 : trouve velo velo_1460 sur station 98
cycliste_3 : va chercher un velo
station_34 : donne le velo velo_540
cycliste_3 : essaie station : 34 
cycliste_3 : trouve velo velo_540 sur station 34
cycliste_4 : va chercher un velo
station_49 : donne le velo velo_725
cycliste_4 : essaie station : 49 
cycliste_4 : trouve velo velo_725 sur station 49
cycliste_5 : va chercher un velo
station_57 : donne le velo velo_890
cycliste_5 : essaie station : 57 
cycliste_5 : trouve velo velo_890 sur station 57
cycliste_6 : va chercher un velo
station_71 : donne le velo velo_1060
cycliste_6 : essaie station : 71 
cycliste_6 : trouve velo velo_1060 sur station 71
cycliste_7 : va chercher un velo
station_81 : donne le velo velo_1215
cycliste_7 :

In [14]:
!rm cyclistes.csv


rm: cyclistes.csv: No such file or directory


In [15]:
path_to_cyclistes = "./cyclistes.csv"
logger_ = Logger(path_to_cyclistes)
infos_voulues = ["id", "sexe", "sportivite", "age", "sur_velo"]
to_write      = ",".join(infos_voulues)
logger_.log(to_write)
for cycliste in paris.cyclistes:
    dico          = cycliste.__dict__        
    infos         = [str(dico[voulue]) for voulue in infos_voulues]
    to_write      = ",".join(infos)
    logger_.log(to_write)
    

In [16]:
logger_.write_to_disk()
